In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
df = pd.read_csv(r'C:\Users\raoma\Documents\michigan\info_vis\group\649_COVID_travel\data\international-travel-covid.csv')
df["Day"] = pd.to_datetime(df["Day"])
codes = pd.read_csv(r'C:\Users\raoma\Documents\michigan\info_vis\group\649_COVID_travel\data\all.csv')
df["id"] = 0
df.rename(columns={'international_travel_controls': 'restrictions'}, inplace=True)

In [2]:
#codes.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [3]:
for index, row in codes.iterrows():
    df.loc[(df['Code'] == row["alpha-3"]),'id'] = row["country-code"]

In [4]:
df['restrictions'] = df['restrictions'].replace([0,1,2,3,4], 
                                        ["No measures","Screening","Quarantine from high-risk regions","Ban on high-risk regions","total border closure"])

In [5]:
#df.head(15)

,Entity,Code,Day,restrictions,id
0,Afghanistan,AFG,2020-01-01,No measures,4
1,Afghanistan,AFG,2020-01-02,No measures,4
2,Afghanistan,AFG,2020-01-03,No measures,4
3,Afghanistan,AFG,2020-01-04,No measures,4
4,Afghanistan,AFG,2020-01-05,No measures,4
5,Afghanistan,AFG,2020-01-06,No measures,4
6,Afghanistan,AFG,2020-01-07,No measures,4
7,Afghanistan,AFG,2020-01-08,No measures,4
8,Afghanistan,AFG,2020-01-09,No measures,4
9,Afghanistan,AFG,2020-01-10,No measures,4


In [6]:
#df.dtypes

Entity                  object
Code                    object
Day             datetime64[ns]
restrictions            object
id                       int64
dtype: object

In [7]:
df['year'] = df.Day.map(lambda x: x.year)
df['month'] = df.Day.map(lambda x: x.month)
df['day'] = df.Day.map(lambda x: x.day)

In [8]:
df_first = df[df["day"] == 1]

In [9]:
#drop anything before 2023
df_first = df_first[(df_first["Day"] < '2023-01-01')]

In [10]:
#df_first

,Entity,Code,Day,restrictions,id,year,month,day
0,Afghanistan,AFG,2020-01-01,No measures,4,2020,1,1
31,Afghanistan,AFG,2020-02-01,No measures,4,2020,2,1
60,Afghanistan,AFG,2020-03-01,Ban on high-risk regions,4,2020,3,1
91,Afghanistan,AFG,2020-04-01,Ban on high-risk regions,4,2020,4,1
121,Afghanistan,AFG,2020-05-01,Ban on high-risk regions,4,2020,5,1
...,...,...,...,...,...,...,...,...
202702,Zimbabwe,ZWE,2022-08-01,Screening,716,2022,8,1
202733,Zimbabwe,ZWE,2022-09-01,Screening,716,2022,9,1
202763,Zimbabwe,ZWE,2022-10-01,Screening,716,2022,10,1
202794,Zimbabwe,ZWE,2022-11-01,Screening,716,2022,11,1


In [47]:
alt.data_transformers.disable_max_rows()

source = alt.topo_feature(data.world_110m.url, "countries")

background = alt.Chart(source).mark_geoshape(fill="white")

years=list(df_first['year'].unique())
years.sort()

selectorYear = alt.selection_single(
    name='Y',
    fields=['year'],
    init={"year":years[0]},
    bind=alt.binding_select(options=years, name="Year: ")
)

months=list(df_first['month'].unique())
months.sort()

selectorMonth = alt.selection_single(
    name='Months',
    fields=['month'],
    init={"month":months[0]},
    bind=alt.binding_select(options=months, name="Month: "),
)

highlight = alt.selection_single(fields=['restrictions'], bind='legend')
opacityCondition = alt.condition(highlight, alt.value(1.0), alt.value(0.2))

foreground = alt.Chart(df_first,title="The COVID restrictions in each country on the 1st of the month").mark_geoshape(
        stroke="black", strokeWidth=0.15
    ).encode(
        alt.Color(
            "restrictions:N",
            scale=alt.Scale(domain=["No measures","Screening","Quarantine from high-risk regions","Ban on high-risk regions","total border closure"],
                            range=['#ffffcc','#fbec5d','#ffbf00','#ff4d00','#e62020']),
            legend=alt.Legend(title="", orient="top")
            
        )
        ,tooltip=[alt.Tooltip('Entity:N', title="Country"), alt.Tooltip('restrictions:N', title="Restrictions")],
        opacity=opacityCondition
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(source, key='id',
                             fields=["type", "properties", "geometry"])
    ).project("naturalEarth1").transform_filter(
    selectorYear & selectorMonth
)

foreground = foreground.add_selection(selectorYear, selectorMonth, highlight).properties(width=700, height=400)
foreground

alt.Chart(...)

In [43]:
alt.renderers.set_embed_options(
    padding={"left": 0, "right": 0, "bottom": 0, "top": 0}
)
selectorYear2 = alt.selection_single(
    name='Years',
    fields=['year'],
    init={"year":years[0]},
    bind=alt.binding_radio(options=years,name="Year: ")
    #bind=alt.binding_select(options=years, name="Year")
)
highlight2 = alt.selection_single(fields=['restrictions'], bind='legend')
opacityCondition = alt.condition(highlight, alt.value(1.0), alt.value(0.2))
monthNames = ["","January","February","March","April","May","June","July","August","September","October","November","December"]
facet = alt.concat(*(
 alt.Chart(df_first[df_first["month"] == month], title=monthNames[month]).mark_geoshape(
        stroke="black", strokeWidth=0.15
    ).encode(
        alt.Color(
            "restrictions:N",
            scale=alt.Scale(domain=["No measures","Screening","Quarantine from high-risk regions","Ban on high-risk regions","total border closure"],
                            range=['#ffffcc','#fbec5d','#ffbf00','#ff4d00','#e62020']),
            legend=alt.Legend(title="", orient="top")
        )
        ,tooltip=[alt.Tooltip('Entity:N', title="Country"), alt.Tooltip('restrictions:N', title="Restrictions")],
        opacity=opacityCondition
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(source, key='id',
                             fields=["type", "properties", "geometry"])
    ).project("naturalEarth1").transform_filter(
    selectorYear2
).add_selection(selectorYear2, highlight2)
for month in range(1,13)
), columns=3
).properties(background = '#f9f9f9',
                    title = alt.TitleParams(text = 'The COVID restrictions throughout the different months of the year')
                  )
facet

alt.ConcatChart(...)

In [13]:
#TODO
#make into streamlit
